In [ ]:


import pandas as pd
import numpy as np
import librosa
# import matplotlib.pylab as plt
# import seaborn as sns
import IPython.display as ipt
import glob
import os
from tinytag import TinyTag
from sklearn.preprocessing import normalize






import kagglehub
path = ""
try:
    path = kagglehub.dataset_download("noahbadoa/fma-dataset-100k-music-wav-files")
    print("Path to dataset files:", path)
except Exception as e:
    print("Error:", e)

target_path = path + r'\fma_large'

files = glob.glob(path + r'\**\*.mp3', recursive = True)

extract_features = []


for i in range(len(files)):
    try: 
        
        tag = TinyTag.get(files[i])
        try:
            y, sr = librosa.load(files[i], sr=22050)
        except Exception as e:
            print(e)
            continue
        
        if y.size == 0 or not np.any(y) or len(y) < 2048:
            print(f"Empty/short audio file skipped")
            continue

        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        if not np.any(onset_env):
            print("No onset detected, skipped")
            continue

        
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)
    
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    
        spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    
        zero_crossing = librosa.feature.zero_crossing_rate(y)

        
        
        # aggregate matrices 
        mfcc_mean = np.mean(mfcc)
        mfcc_std = np.std(mfcc)
        centroid_mean = np.mean(centroid)

        normalize_feats = [mfcc_mean, mfcc_std, tempo[0], centroid_mean,  np.mean(spec_rolloff), np.mean(chroma), np.mean(zero_crossing)]
        feats_normalized = normalize(np.array(normalize_feats).reshape(1, -1), norm='l2')

        
        
        extract_features.append({"mfcc_mean": feats_normalized[0][0],
        "mfcc_std": feats_normalized[0][1],
        "tempo": feats_normalized[0][2],  # tempo is already an array of 1
        "spec_centroid_mean": feats_normalized[0][3],
        "spec_rolloff_mean": feats_normalized[0][4],
        "chroma_mean": feats_normalized[0][5],
        "zero_crossing_mean": feats_normalized[0][6],
        "title": tag.title,
        "artist": tag.artist,
        "genre": tag.genre,
        "duration": tag.duration,
        "index": i})
    except Exception as e: 
        print(e)
        continue

feature_df = pd.DataFrame(extract_features)

feature_df.to_csv("featuresV3.csv")

print(feature_df)



In [ ]:
import pandas as pd 



df = pd.read_csv(r'D:\songrec\songrec_metadata\fma_metadata\raw_artists.csv')

pd.set_option('display.max_rows', None)


In [50]:
from sklearn.neighbors import NearestNeighbors as nn 
import pandas as pd
import numpy as np 
import glob
import IPython.display as ipt
import librosa
import kagglehub

# KNN
df = pd.read_csv('featuresV2.csv', usecols=range(7))
test = np.array(df.iloc[15000]).reshape(1,-1)
data = np.array(df.values.tolist())
knn = nn(n_neighbors=10, algorithm='kd_tree').fit(data)
distance, indeces = knn.kneighbors(test)

path = ""
try:
    path = kagglehub.dataset_download("noahbadoa/fma-dataset-100k-music-wav-files")
    print("Path to dataset files:", path)
except Exception as e:
    print("Error:", e)

target_path = path + r'\fma_large'
files = glob.glob(path + r'\**\*.mp3', recursive = True)

y, sr = librosa.load(files[indeces[0][3]], sr=22050)
ipt.Audio(data=y, rate=sr)

Path to dataset files: C:\Users\John Pork\.cache\kagglehub\datasets\noahbadoa\fma-dataset-100k-music-wav-files\versions\1


In [51]:
import numpy as np 

class EpsilonGreedy:

    def __init__(self, arms, epsilon):
        self.arms = arms
        self.epsilon = epsilon
        self.choice_count = np.zeroes(arms)
        self.reward = np.zeroes(arms)

    def choose_arm(self):
        if self.epsilon > np.random.rand():
            return np.random.randint(0, self.arms)
        else:
            return np.argmax(self.reward)

    def update(self, chosen_arm, reward):
        self.choice_count[chosen_arm] += 1

        n = self.choice_count[chosen_arm]
        value = self.reward[chosen_arm]
        self.reward[chosen_arm] += reward


# RL can work well with song recommendations contigent upon the context of the arms chosen
# i dont think that KNN will be useful for the purposes of shrinking the pool of arms
# im thinking we need some other algo to give us a consistent pool of arms, probably clustering
# once we discover what a user likes, we wont jump clusters much is my hypothesis, so we will have a consistent pool of arms
    